In [14]:
# set random seed
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Parameters

In [15]:
# import pyepo
# generate data
grid = (5,5) # grid size
num_data = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.2 # polynomial degree
e = 0.5 # noise width

# Generate Data

In [16]:
from Data import data_generation
data_gen = data_generation()

#  ****** Data generation process is the same as SPO+ *********
feats, costs = data_gen.generate_Shortest_Path_Data(num_data+num_test, num_feat, grid, deg, e, seed=42)

#  ****** Data generation process is the same as DDR *********
# lower = 0
# upper = 1
# p = 5
# d = 40
# alpha = 1
# mis = 4
# n_epsilon = 1
# W_star = data_gen.generate_truth("",lower, upper, p, d, version = 0) 
# x_test, z_test_ori, c_test, x_train, z_train_ori, c_train, W_star = data_gen.generate_samples("",p, d, num_test, num_data, alpha, W_star, n_epsilon, mis, thres = 10, 
#                         version = 1, x_dist = 'normal', e_dist = 'normal', x_low = 0, x_up = 2, x_mean = 2, x_var = 0.25, bump = 0) 

# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test, random_state=42)

### SPO+ 

In [17]:
from SPO_Plus import run_SPO_Shortest_Path
SPO_runner = run_SPO_Shortest_Path()
batch_size = 20
num_epochs = 1
arcs,loader_train,loader_test,cost_Oracle,cost_SPO = SPO_runner.run(x_train,c_train,x_test,c_test,batch_size,num_feat,grid,num_epochs,True)

Test
Optimizing for optDataset...


100%|██████████| 100/100 [00:00<00:00, 1416.89it/s]


Test
Optimizing for optDataset...


100%|██████████| 1000/1000 [00:00<00:00, 2093.28it/s]


oracle cost =  6.6683534908294675
Num of cores: 2
epoch 0: Average SPO Cost =  8.279449820011854
epoch =  0  Average SPO Cost =  8.206841227144002


## Inner problem of the shortest path problem

In [18]:
def solve_Inner_Problem(arcs,cost_true,cost_pred,mu,is_binary):
    """
    A method to build Gurobi model

    Returns:
        tuple: optimization model and variables
    """
    import gurobipy as gp
    from gurobipy import GRB
    # import gurobipy as grb
    # ceate a model
    m = gp.Model("shortest path")
    m.setParam('OutputFlag', 0)
    if is_binary:
    # varibles
        x = m.addVars(arcs, lb = 0,vtype=GRB.BINARY  ,name="x")
    else:
        x = m.addVars(arcs, lb = 0,name="x")
    # sense
    # m.modelSense = GRB.MINIMIZE
    # flow conservation constraints
    for i in range(grid[0]):
        for j in range(grid[1]):
            v = i * grid[1] + j
            expr = 0
            for e in arcs:
                # flow in
                if v == e[1]:
                    expr += x[e]
                # flow out
                elif v == e[0]:
                    expr -= x[e]
            # source
            if i == 0 and j == 0:
                m.addConstr(expr == -1)
            # sink
            elif i == grid[0] - 1 and j == grid[0] - 1:
                m.addConstr(expr == 1)
            # transition
            else:
                m.addConstr(expr == 0)
    m.setObjective( sum([-mu*cost_true[ind] * x[arcs[ind]] - (1-mu)*cost_pred[ind] * x[arcs[ind]] for ind in range(len(arcs))]) , GRB.MAXIMIZE)
    m.optimize()
    sol = m.getAttr('x')
    # print("sol = ",sol)
    # shortest_path = obtain_path(arcs,sol)
    obj = m.ObjVal
    # print("shortest_path = ",shortest_path)
    return obj,sol

## The dual problem of the inner problem

In [19]:
def solve_Inner_Problem_Dual(arcs,cost_true,cost_pred,mu):
    num_nodes = 25
    import gurobipy as gp
    from gurobipy import GRB
    # import gurobipy as grb
    # ceate a model
    m = gp.Model("Maximium path")
    m.setParam('OutputFlag', 0)
    alpha = m.addVars(num_nodes,name="alpha")
    # sense
    # m.modelSense = GRB.MINIMIZE
    # flow conservation constraints
    for ind in range(len(arcs)):
        e = arcs[ind]
        j = e[1]
        i = e[0]
        # print("j = ",j,", i = ",i, ", e = ",e)
        m.addConstr(alpha[j] - alpha[i] >= -mu*cost_true[ind] - (1-mu)*cost_pred[ind])

    m.setObjective( alpha[num_nodes-1] - alpha[0], GRB.MINIMIZE)
    m.optimize()
    # # print("sol = ",sol)
    obj = m.ObjVal
    # print("obj = ",obj)
    return obj

# Check the equivalence between shortest path, its linear relaxation and dual problem

In [20]:
cost_true_tem = np.random.uniform(0,1,40) 
cost_pred_tem = np.random.uniform(0,1,40) 
mu_fixed = 0.4
obj_continous,sol_continous = solve_Inner_Problem(arcs,cost_true_tem,cost_pred_tem,mu_fixed,False)
print("obj_continous = ",obj_continous)

obj_binary,sol_binary = solve_Inner_Problem(arcs,cost_true_tem,cost_pred_tem,mu_fixed,True)
print("obj_binary = ",obj_binary)


obj_dual = solve_Inner_Problem_Dual(arcs,cost_true_tem,cost_pred_tem,mu_fixed)
print("obj_dual = ",obj_dual)


obj_continous =  -2.224103448996429
obj_binary =  -2.224103448996429
obj_dual =  -2.224103448996429
